In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import fpfs
import fitsio
import impt
import jax.numpy as jnp
cov_fname = os.path.join(impt.fpfs.__data_dir__, "modes_cov_mat_paper3_045.fits")
cov_mat = fitsio.read(cov_fname)

In [2]:
test_fname = os.path.join(
    impt.fpfs.__data_dir__,
    "fpfs-cut32-0000-g1-0000.fits",
)
test_fname = "/lustre/work/xiangchong.li/work/FPFS2/sim/srcfs3_unif3_cosmo085-var7em3_try3/psf60/fpfs-cut32-0000-g1-0000.fits"


# impt.fpfs
cat = impt.fpfs.read_catalog(test_fname)
# FPFS
data = fitsio.read(test_fname)
cov_mat2 = impt.fpfs.utils.fpfscov_to_imptcov(data)

2023/01/11 21:49:47 ---  Remote TPU is not linked into jax; skipping remote TPU.
2023/01/11 21:49:47 ---  Unable to initialize backend 'tpu_driver': Could not initialize backend 'tpu_driver'
2023/01/11 21:49:47 ---  Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023/01/11 21:49:47 ---  Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023/01/11 21:49:47 ---  Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
2023/01/11 21:49:47 ---  Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
2023/01/11 21:49:47 ---  No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [20]:
use_sig = False
funcnm = "ts2"
params = impt.fpfs.FpfsParams(
    Const = 2.0,
    lower_m00=1.5,
    sigma_m00=0.5,
    lower_r2=-0.04,
    upper_r2=1000.,
    sigma_r2=0.5,
    sigma_v=0.3,
)
e1_impt = impt.fpfs.FpfsE1(params, func_name=funcnm)
e2_impt = impt.fpfs.FpfsE2(params, func_name=funcnm)
w_sel = impt.fpfs.FpfsWeightSelect(params, func_name=funcnm)

e1_s = e1_impt*w_sel
e2_s = e2_impt*w_sel

de1s_dg1 = impt.RespG1(e1_s)
de2s_dg2 = impt.RespG2(e2_s)

In [21]:
selnm = np.array(["M00", "R2"])
#selnm = np.array([])
ell_fpfs = fpfs.catalog.fpfs_m2e(data, const=params.Const, noirev=False)
fs = fpfs.catalog.summary_stats(data, ell_fpfs, use_sig=use_sig, ratio=1.0)
fs = impt.fpfs.test_utils.initialize_FPFS(fs, selnm, params)

ell_fpfs2 = fpfs.catalog.fpfs_m2e(data, const=params.Const, noirev=True)
fs2 = fpfs.catalog.summary_stats(data, ell_fpfs2, use_sig=use_sig, ratio=1.0)
fs2 = impt.fpfs.test_utils.initialize_FPFS(fs2, selnm, params)

In [22]:
print(fs.sumE1)
print(jnp.sum(e1_s.evaluate(cat)))

-96.95327795524238
-96.95327795524237


In [23]:
print(fs.sumE2)
print(jnp.sum(e2_s.evaluate(cat)))

13.077153931073525
13.07715393107352


In [24]:
bnoise = impt.BiasNoise(e1_s, cov_mat2)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumE1 - fs.sumE1 + fs2.corE1 - fs.corE1)

6.124377753137555
-2.179042088071438


In [25]:
bnoise = impt.BiasNoise(e1_s, cov_mat)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumE1 - fs.sumE1 + fs2.corE1 - fs.corE1)

6.1243773928948135
-2.179042088071438


In [26]:
bnoise = impt.BiasNoise(e2_s, cov_mat2)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumE2 - fs.sumE2 + fs2.corE2 - fs.corE2)

1.2053385435752706
2.490314155056273


In [27]:
bnoise = impt.BiasNoise(e2_s, cov_mat)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumE2 - fs.sumE2 + fs2.corE2 - fs.corE2)

1.205338839500813
2.490314155056273


In [28]:
print(fs.sumR1 + fs.corR1)
print(jnp.sum(de1s_dg1.evaluate(cat)))

5630.1219380734665
5619.13419544608


In [29]:
print(fs.sumR2 + fs.corR2)
print(jnp.sum(de2s_dg2.evaluate(cat)))

5709.679293750446
5720.667329781878


In [30]:
bnoise = impt.BiasNoise(de1s_dg1, cov_mat2)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR1 - fs.sumR1 + fs2.corR1 - fs.corR1)

199.66960415224008
-258.5684736308491


In [31]:
bnoise = impt.BiasNoise(de1s_dg1, cov_mat)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR1 - fs.sumR1 + fs2.corR1 - fs.corR1 )

200.4390466963523
-258.5684736308491


In [32]:
bnoise = impt.BiasNoise(de2s_dg2, cov_mat2)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR2 - fs.sumR2 + fs2.corR2 - fs.corR2)

340.1220605236722
-258.4338264163649


In [33]:
bnoise = impt.BiasNoise(de2s_dg2, cov_mat)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR2 - fs.sumR2 + fs2.corR2 - fs.corR2 )

339.3977672891429
-258.4338264163649


In [34]:
cov_mat3 = cov_mat
cov_mat3[8:, 8:] = 0.
bnoise = impt.BiasNoise(de2s_dg2, cov_mat3)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR2 - fs.sumR2 + fs2.corR2 - fs.corR2 )

339.3977672891429
-258.4338264163649


In [35]:
cov_mat3[0,0]

0.2527604999377557

In [36]:
cov_mat2[0,0]

Array(0.2527605, dtype=float64)